In [1]:
import pandas as pd
%matplotlib inline
import urllib
import vzta
import vzta.feature_extractors as fe
import vzta.algorithms as algorithm



In [17]:
from spacy.symbols import *
from itertools import takewhile
#text='your automated system is not answering my questions!!! at\u0026t is looking better than you'.decode('unicode-escape')
text="when does my contract expire?"
#text="why did my bill go up?"
##text ="make a payment arrangement"
doc= vzta.nlp.getDoc(text,parse=True)
for d in doc:
    print( d.orth_, d.pos_, d.dep_, d.head.orth_)
OBJECTS=[dobj, iobj, pobj]


def dedup(l):
    result=[]
    for i in l:
        if i in result:
            continue
        result.append(i)
    return result

def isNegated(v):
    for c in v.children:
        if c.dep==neg:
            return True
    return False

def getPart(v):
    for c in v.rights:
        if c.pos==PART or c.dep==acomp:
            return c
    return None

def get_span_for_compound_noun(noun):
    """
    Return document indexes spanning all (adjacent) tokens
    in a compound noun.
    """
    min_i = noun.i - sum(1 for _ in takewhile(lambda x: x.dep_ in ('compound','amod'),
                                              reversed(list(noun.lefts))))
    return (min_i, noun.i)

def getPreps(deps):
    objs = []
    for dep in deps:
        if dep.pos == ADP and dep.dep == prep:
            objs.extend(
                [tok for tok in dep.rights if tok.dep in OBJECTS or tok.pos == PRON])
    return objs



np_labels = set([nsubj, nsubjpass,dobj, iobj, pobj,attr,dep])
#np_labels = set([dobj, iobj, pobj])
def iter_nps(doc):
    for word in doc:
        if word.dep in np_labels:
            yield [word.head,word.subtree,word.dep]

def isq(doc):
    first_token=doc[0].text.lower()
    last_token=doc[-1].text.lower()
    if first_token in ['when','what','how','why']:
        if first_token=='how':
            if doc[1].text=='much':
                first_token ='how much'
            elif doc[1].text=='many':
                first_token ='how many'
        return first_token
    if last_token=='?':
        return '?'
    return None
def intent(doc):
    result=[]
    
    for sent in doc.sents:
        itr= iter_nps(sent)
        verbs=[]
        intents=[]
        nvintents=[]
        negations=False
        word=None
        nverbs=[]
        for p in itr:
            intent=[t for t in p[1] if t.pos in [VERB,NOUN,PROPN,ADJ,ADV,X] and not t.text.lower() in ['what','much','many','that','my', 'your','I',"'m"]]
            word= p[0]
            if intent: #and word.pos==VERB:  #and and word.lemma_!='be'
                preps = getPreps(word.children)
                intent =intent +[pr for pr in preps]
                verb= []
                negated=False
                if word.pos==VERB:
                    negated = isNegated(word)
                    part = getPart(word)
                    verb.append(word)
                    if part:
                        verb.append(part)
                    verbs.extend(verb)
                    intents.extend(intent)
                else:
                    v=[]
                    if word.dep==prep:
                        if word.head.pos=='VERB':
                            verb.append(word.head)
                        else:
                            min_i,max_i = get_span_for_compound_noun(word.head)
                            v =[doc[min_i:max_i+1]]
                    elif word.dep==prt:
                        verb.extend([word.head,word])
                    elif word.pos in(NOUN,PROPN):
                        min_i,max_i=get_span_for_compound_noun(word)
                        v =[doc[min_i:max_i+1]]
                    else:
                        v=[word]
                    nverbs.extend(v)
                    nvintents.extend(intent)
                
                negations = negations or negated
                
        if intents or nvintents:
                if verbs:
                    del nverbs[:]
                else:
                    verbs.extend(nverbs)
                    del nverbs[:]
                    
                if not intents:
                    intents.extend(nvintents)
                q = isq(sent)
                r =[sent,negations,verbs,nverbs,intents,q]
                result.append(r)
               

    return result
        
    return result
def intent_extractor(text):
    txt=text.decode('utf-8').strip()
    #txt = re.sub(r'(?!\\+u00)','',txt)
    #txt=unicode(txt)
    try:
        txt =txt.decode('unicode-escape')
        txt = txt.replace('&','and')

    except:
        txt = txt.replace('\\','')
    doc = vzta.nlp.getDoc(txt,parse=True)
    result=[]
    if len(doc)>3:
        result = intent(doc)
        
    return result


#intent_extractor('why do i have to pay an upgrade fee?. ok. ok. Larry Zimmerman.  ok. we want to get the iphone on 410-000-000. mine is 410.000.000. i am Larry. i was looking at do a contract and pay up front too. it looks like $299. and equipment protection')
        


def getIntents(text):
    res=[]
    result={}
    intents= intent_extractor(text)
    for i in intents:
        result['sent']= i[0].text
        if r[1]:
                result['negation']='!'
        else:
                result['negation']=''
        
        actions = i[2]
        result['action'] = ' '.join(dedup([a.text for a in actions]))
        result['subject']=' '.join(dedup([a.text for a in i[3]]))
        q= i[4]
        if q:
                result['Q']=q
        else:
               result['Q'] =''
        res.append(result)
    return res

intent_extractor(text)
     

(u'when', u'ADV', u'advmod', u'expire')
(u'does', u'VERB', u'aux', u'expire')
(u'my', u'ADJ', u'poss', u'contract')
(u'contract', u'NOUN', u'nsubj', u'expire')
(u'expire', u'VERB', u'ROOT', u'expire')
(u'?', u'PUNCT', u'punct', u'expire')


[[when does my contract expire?, False, [expire], [], [contract], u'when']]

In [15]:
search_file=r"C:\Users\v617346\Documents\TA\data\search.txt"
search_corpus = vzta.corpus.Corpus(search_file)
search_corpus=search_corpus[0]
original=[]
sents=[]
negations=[]
verbs=[]
nverbs=[]
intents=[]
Qs=[]

#[sent,negated,verb,intent,q]

for text in search_corpus:
    result= intent_extractor(text)
    if result:
        for r in result:
            original.append(text)
            sents.append(r[0].text)
            if r[1]:
                negations.append('!')
            else:
                negations.append('')
            actions = r[2]
            verbs.append( ' '.join(dedup([a.text for a in actions])))
            
            nverbs.append( ' '.join(dedup([a.text for a in r[3]])))
            
            intents.append( ' '.join(dedup([a.text for a in r[4]])))
            q= r[5]
            if q:
                Qs.append(q)
            else:
                Qs.append('')
            
data = {'Original': original,'Sentence': sents,'Negation':negations,'Action':verbs, 'NV Action':nverbs, 'Subject':intents, 'Question':Qs}
dframe = pd.DataFrame(data, columns=data.keys())

C:\Users\v617346\Documents\TA\data\search.txt


In [16]:
dframe.to_excel(r'C:\Users\v617346\Documents\TA\intent_mappings7.xlsx')